In [3]:
from cassandra.cluster import Cluster
import json
import time

# Koneksi ke Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Buat keyspace (jika belum)
session.execute("""
CREATE KEYSPACE IF NOT EXISTS Tubes_Orbid
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

# Tunggu sebentar agar keyspace siap (menghindari error 'does not exist')
time.sleep(2)

In [6]:
session.set_keyspace('tubes_orbid')

In [7]:
# Buat tabel (ganti struktur sesuai dataset kamu)
session.execute("""
CREATE TABLE IF NOT EXISTS data_karyawan (
    id_karyawan text PRIMARY KEY,
    nama_lengkap text,
    umur int,
    jenis_kelamin text,
    posisi text,
    jam_kerja_per_bulan int
)
""")

In [9]:
# Baca file JSON
with open("indomaret_employees.json", "r") as file:
    data = json.load(file)

In [10]:
# Insert setiap record ke Cassandra
for row in data:
    session.execute("""
    INSERT INTO data_karyawan (
        id_karyawan, nama_lengkap, umur,
        jenis_kelamin, posisi, jam_kerja_per_bulan
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['id_karyawan'],
        row['nama_lengkap'],
        int(row['umur']),
        row['jenis_kelamin'],
        row['posisi'],
        int(row['jam_kerja_per_bulan'])
    ))

In [13]:
rows = session.execute("SELECT * FROM data_karyawan;")
for row in rows:
    print(row)

Row(id_karyawan='P006', jam_kerja_per_bulan=242, jenis_kelamin='Perempuan', nama_lengkap='Indah Permatasari', posisi='Staf Gudang', umur=39)
Row(id_karyawan='P016', jam_kerja_per_bulan=235, jenis_kelamin='Perempuan', nama_lengkap='Nisa Rahmawati', posisi='Keamanan', umur=38)
Row(id_karyawan='P007', jam_kerja_per_bulan=138, jenis_kelamin='Laki-laki', nama_lengkap='Wahyu Hidayat', posisi='Supervisor', umur=52)
Row(id_karyawan='P019', jam_kerja_per_bulan=197, jenis_kelamin='Laki-laki', nama_lengkap='Bayu Nugroho', posisi='Supervisor', umur=32)
Row(id_karyawan='P010', jam_kerja_per_bulan=130, jenis_kelamin='Laki-laki', nama_lengkap='Dewi Ayu Lestari', posisi='Petugas Kebersihan', umur=43)
Row(id_karyawan='P018', jam_kerja_per_bulan=139, jenis_kelamin='Laki-laki', nama_lengkap='Intan Prameswari', posisi='Kasir', umur=53)
Row(id_karyawan='P011', jam_kerja_per_bulan=135, jenis_kelamin='Laki-laki', nama_lengkap='Fajar Ramadhan', posisi='Keamanan', umur=34)
Row(id_karyawan='P015', jam_kerja_per

Sebelum index

In [14]:
# Query tanpa index
query = "SELECT * FROM data_karyawan WHERE posisi = 'Kasir' ALLOW FILTERING"

# Ukur waktu sebelum indexing
start = time.time()
rows = session.execute(query)
result_before = list(rows)
end = time.time()

print("⏱️ Waktu SEBELUM indexing:", end - start, "detik")
print(f"📋 Jumlah hasil: {len(result_before)}")


⏱️ Waktu SEBELUM indexing: 0.017951250076293945 detik
📋 Jumlah hasil: 3


In [15]:
session.execute("CREATE INDEX IF NOT EXISTS posisi_index ON data_karyawan (posisi);")

Sesudah index

In [18]:
query = "SELECT * FROM data_karyawan WHERE posisi = 'Kasir' ALLOW FILTERING"

start = time.time()
rows = session.execute(query)
result_after = list(rows)
end = time.time()

print("⏱️ Waktu SESUDAH indexing:", end - start, "detik")
print(f"📋 Jumlah hasil: {len(result_after)}")

⏱️ Waktu SESUDAH indexing: 0.0169525146484375 detik
📋 Jumlah hasil: 3
